In [ ]:
!pip install transformers -q

In [54]:
from transformers import BertModel, BertTokenizerFast #base bert
from transformers import AutoModel, AutoTokenizer  #bertweet
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaTokenizerFast  #roberta
import torch
import torch.nn as nn
# from sklearn.data
from pprint import pprint
import re
from collections import Counter
import pandas as pd
from tqdm import tqdm

In [55]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [79]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    path = 'drive/MyDrive/Colab Notebooks/NLP/final_project/'
except Exception:
    path= ''

In [80]:
mode = 0 #mode 0(original data) or 1(pre-processed data)
base_loc = 'Project_4_datasets/Project_2_datasets/Task 1/'
if mode==0:
    #read from original data
    train_loc = path+ base_loc +'task2_en_training.tsv'
    test_loc = path+ base_loc +'task2_en_validation.tsv'
    train_data = pd.read_table(train_loc)
    test_data = pd.read_table(test_loc)
else:
    #read from pre-processed data
    train_loc = path+ base_loc + 'pre_task2_en_training.csv'
    test_loc = path+ base_loc + 'pre_task2_en_validation.csv'
    train_data = pd.read_csv(train_loc)
    test_data = pd.read_csv(test_loc)

# pd.read_t

In [81]:
#trying different combination of input data.
#this is for undersampling and oversampling
cnt = sum(train_data['class']==1)
pos = train_data[train_data['class']==1].copy()
# pos.concat(train_data.sample(cnt))
pos = pd.concat([pos, train_data.sample(2*cnt)])

"""cnt = sum(train_data['class']==1)
pos = train_data[train_data['class']==1].copy()
 pos.concat(train_data.sample(cnt))
pos = pd.concat([pos, train_data.sample(4*cnt)])
pos = pd.concat([pos, pos])"""

"cnt = sum(train_data['class']==1)\npos = train_data[train_data['class']==1].copy()\n pos.concat(train_data.sample(cnt))\npos = pd.concat([pos, train_data.sample(4*cnt)])\npos = pd.concat([pos, pos])"

In [82]:
train_data
# test_data

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make thin..."
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made ch...
4,345567138376994816,138795534,0,have decided to skip my #humira shot today. my...
...,...,...,...,...
20539,469009954251481088,23177032,1,@hornetweb my mri scan shows when it happens b...
20540,468518427125358592,1961096371,1,remember 2003. vioxx costs $65m to ontario dru...
20541,512254891361075200,321391071,1,asians are at higher risk for severe allergic ...
20542,491775200610893825,2484689840,1,5. so what caused the #estrogen surges in #nuv...


In [83]:
# to asign sampled data to train_data; Use only when sampling the data
train_data = pos
train_data.reset_index(drop=True, inplace=True)

In [84]:
# test_data
train_data.shape

(5709, 4)

In [85]:
def preprocess(data: pd.DataFrame, col_name: str):
    for i in tqdm(range(len(data[col_name]))):
        text = data[col_name][i]
        text = text.lower()
        if text[:3] == 'rt ':
            text = text[3:]
#         mention_pattern = "@[\w]+"
#         replace_mention = '[user]'
#         text = re.sub(mention_pattern, replace_mention, text)
#         url_pattern = "http(.)*"
#         replace_url = "[url]"
#         text = re.sub(url_pattern, replace_url, text)
#         data[col_name][i] = text

In [86]:
preprocess(train_data, 'tweet')
preprocess(test_data, 'tweet')

100%|██████████| 5134/5134 [00:00<00:00, 39388.54it/s]


In [ ]:
#to save the preprocesss tweets; to save future preprocess operations
train_data.to_csv(path_or_buf=path+base_loc+'pre_task2_en_training.csv', index=False)
test_data.to_csv(path_or_buf=path+base_loc+'pre_task2_en_validation.csv', index=False)

In [ ]:
# test_data
train_data['class'].shape

In [60]:
# test_data.columns
train_data.columns

Index(['tweet_id', 'user_id', 'class', 'tweet'], dtype='object')

In [87]:
"""This contains various tokenizer, uncomment and use the tokenizer as per the model requirement"""

# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [91]:
#this will give train and test data encodings.
train_enc = tokenizer.batch_encode_plus(train_data['tweet'].to_list(), padding="longest", truncation=True, return_tensors="pt")
test_enc = tokenizer.batch_encode_plus(test_data['tweet'].to_list(), padding="longest", truncation=True, return_tensors="pt")

In [89]:
test_enc.keys()

dict_keys(['input_ids', 'attention_mask'])

In [90]:
"""use this if working on BERT, Bertweet; token_type_ids is additional field"""
train_input_ids, train_type_ids, train_attention_mask = train_enc['input_ids'], train_enc['token_type_ids'], train_enc['attention_mask']
test_input_ids, test_type_ids, test_attention_mask = test_enc['input_ids'], test_enc['token_type_ids'], test_enc['attention_mask']

KeyError: 'token_type_ids'

In [92]:
"""use this if working on RoBERTa; token_type_ids is not generated in this"""
#RobertA
train_input_ids, train_attention_mask = train_enc['input_ids'], train_enc['attention_mask']
test_input_ids, test_attention_mask = test_enc['input_ids'], test_enc['attention_mask']
train_input_ids.shape, train_attention_mask.shape

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Below we define a function to create train, test & valid dataloaders in Pytorch

batch_size = 32

def get_dataloader_roberta(input_ids, attn_mask, y):
    data = TensorDataset(input_ids, attn_mask, y)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

train_datalaoder = get_dataloader_roberta(train_input_ids, 
                                          train_attention_mask, 
                                          torch.tensor(train_data['class']))
test_datalaoder = get_dataloader_roberta(test_input_ids, test_attention_mask, torch.tensor(test_data['class']))

In [66]:
"""check the data shape; uncomment appropriate code as per the model"""

train_input_ids.shape, train_type_ids.shape, train_attention_mask.shape
# train_input_ids.shape, train_attention_mask.shape

(torch.Size([20544, 74]), torch.Size([20544, 74]), torch.Size([20544, 74]))

In [67]:
#use this when running BERT or RoBERTa
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Below we define a function to create train, test & valid dataloaders in Pytorch

batch_size = 32

def get_dataloader(input_ids, type_ids, attn_mask, y):
    data = TensorDataset(input_ids, type_ids, attn_mask, y)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

train_datalaoder = get_dataloader(train_input_ids, train_type_ids, train_attention_mask, torch.tensor(train_data['class']))
test_datalaoder = get_dataloader(test_input_ids, test_type_ids, test_attention_mask, torch.tensor(test_data['class']))

In [93]:
# Sanity check that the tensors returned by the dataloader are correct

"""for batch in train_datalaoder:
    input_ids, type_ids, attn_mask, y = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, y.shape)
    break
"""

#Roberta
for batch in train_datalaoder:
    input_ids,attn_mask, y = batch
    print(input_ids.shape, attn_mask.shape, y.shape)
    break

torch.Size([32, 73]) torch.Size([32, 73]) torch.Size([32])


In [94]:
#this class deals with BERT & B model
class BERTClassifier(nn.Module):
  def __init__(self, transformer):
    super().__init__()
    self.transformer = transformer
    self.linear = nn.Linear(768, 2)
  
  def forward(self, in_ids, type_ids, attn_mask):
    op = self.transformer(input_ids=in_ids, attention_mask=attn_mask, 
                          token_type_ids=type_ids)
    
    return  self.linear(op["pooler_output"])

In [95]:
class ROBERTaClassifier(nn.Module):
  def __init__(self, transformer):
    super(ROBERTaClassifier, self).__init__()
    self.transformer = transformer
    self.linear = nn.Linear(768, 2)
  
  def forward(self, in_ids, attn_mask):
    op = self.transformer(input_ids=in_ids, attention_mask=attn_mask)
    
    return  self.linear(op["pooler_output"])

In [96]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [97]:
#BERT
"""transformer = BertModel.from_pretrained("bert-base-uncased")
model = BERTClassifier(transformer).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')"""

#bertweet
"""transformer = AutoModel.from_pretrained("vinai/bertweet-base")
model = BERTClassifier(transformer).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')
"""
#ROBERTa
# transformer = RobertaConfig()
transformer = RobertaModel.from_pretrained("roberta-base")
model = ROBERTaClassifier(transformer).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 124,647,170 trainable parameters


In [98]:
for name, param in model.named_parameters():
  if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
    param.requires_grad = True
  else:
    param.requires_grad = False
  print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

transformer.embeddings.word_embeddings.weight torch.Size([50265, 768]) False
transformer.embeddings.position_embeddings.weight torch.Size([514, 768]) False
transformer.embeddings.token_type_embeddings.weight torch.Size([1, 768]) False
transformer.embeddings.LayerNorm.weight torch.Size([768]) False
transformer.embeddings.LayerNorm.bias torch.Size([768]) False
transformer.encoder.layer.0.attention.self.query.weight torch.Size([768, 768]) False
transformer.encoder.layer.0.attention.self.query.bias torch.Size([768]) False
transformer.encoder.layer.0.attention.self.key.weight torch.Size([768, 768]) False
transformer.encoder.layer.0.attention.self.key.bias torch.Size([768]) False
transformer.encoder.layer.0.attention.self.value.weight torch.Size([768, 768]) False
transformer.encoder.layer.0.attention.self.value.bias torch.Size([768]) False
transformer.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768]) False
transformer.encoder.layer.0.attention.output.dense.bias torch.Size(

In [99]:
# Training function: Performs forward propagation, backpropagation & optimization.
# We also implement gradient clipping, which prevents the gradients from exploding
#BERT, Bertweet
def train(model, dataloader, optimizer, criterion, clip=1.0):

    model.train()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for ix, batch in tqdm(enumerate(dataloader)):
        batch = tuple(t.to(device) for t in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        optimizer.zero_grad()
        output = model(input_ids, type_ids, attn_mask)     
        loss = criterion(output, y)
        loss.backward()

        #gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        ep_t_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        tgt.extend(y.tolist())

    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro')

# Evaluation function: Calculates loss on the validation data.
from sklearn import metrics

def evaluate(model, dataloader, criterion):

    model.eval()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for ix, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, type_ids, attn_mask, y = batch

        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            
            loss = criterion(output, y)

            ep_t_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            tgt.extend(y.tolist())
        
    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro'), pred, tgt

In [100]:
# Training function: Performs forward propagation, backpropagation & optimization.
# We also implement gradient clipping, which prevents the gradients from exploding
#RoBERTa
def train_roberta(model, dataloader, optimizer, criterion, clip=1.0):

    model.train()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for ix, batch in tqdm(enumerate(dataloader)):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attn_mask, y = batch
        
        optimizer.zero_grad()
        output = model(input_ids, attn_mask)     
        loss = criterion(output, y)
        loss.backward()

        #gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        ep_t_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        tgt.extend(y.tolist())

    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro')

# Evaluation function: Calculates loss on the validation data.
from sklearn import metrics

def evaluate_roberta(model, dataloader, criterion):

    model.eval()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for ix, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attn_mask, y = batch

        with torch.no_grad():
            output = model(input_ids, attn_mask)
            
            loss = criterion(output, y)

            ep_t_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            tgt.extend(y.tolist())
        
    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro'), pred, tgt

In [103]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

In [104]:
best_valid_loss = float('inf')
tot_t_loss, tot_v_loss =[],[]
N_EPOCHS = 10

In [77]:
#BERT, Birtweet
import time
from tqdm import tqdm
t1 = time.time()
try:
    for epoch in tqdm(range(N_EPOCHS)): 

        tr_l, tr_f1= train(model, train_datalaoder, optim, criterion)
        tot_t_loss.append(tr_l)

        val_l, val_f1, pred, tgt = evaluate(model, test_datalaoder, criterion)
        tot_v_loss.append(val_l)

        if val_l < best_valid_loss:
            best_valid_loss = val_l
            best_pred, best_tgt = pred, tgt
            torch.save(model.state_dict(), 'model_least_loss.pt')
            print("\nBest Model Saved !!")
        elif epoch % 3 == 0:
            torch.save(model.state_dict(), 'model_checkpoint_'+str(epoch)+'.pt')
            print("\Checkpoint Model Saved !!")
        print("\n")
        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Total Loss: {tr_l:.3f} | Train F1: {tr_f1:.3f}')
        print(f'\tVal. Total Loss: {val_l:.3f} | Valid F1: {val_f1:.3f}')
        print("_________________________________________________________________")
except:
    t2 = time.time()
t2 = time.time()
print(f"total time taken: {(t2-t1)/60}")

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
2it [00:00, 11.28it/s]
4it [00:00, 12.27it/s]
6it [00:00, 12.37it/s]
8it [00:00, 13.16it/s]
10it [00:00, 12.84it/s]
12it [00:00, 13.40it/s]
14it [00:01, 13.21it/s]
16it [00:01, 13.42it/s]
18it [00:01, 13.46it/s]
20it [00:01, 13.24it/s]
22it [00:01, 13.79it/s]
24it [00:01, 13.33it/s]
26it [00:01, 13.49it/s]
28it [00:02, 13.16it/s]
30it [00:02, 13.85it/s]
32it [00:02, 13.42it/s]
34it [00:02, 13.88it/s]
36it [00:02, 13.37it/s]
38it [00:02, 13.72it/s]
40it [00:02, 13.67it/s]
42it [00:03, 13.51it/s]
44it [00:03, 13.76it/s]
46it [00:03, 13.47it/s]
48it [00:03, 13.72it/s]
50it [00:03, 13.34it/s]
52it [00:03, 14.00it/s]
54it [00:03, 13.52it/s]
56it [00:04, 13.93it/s]
58it [00:04, 13.41it/s]
60it [00:04, 13.77it/s]
62it [00:04, 13.70it/s]
64it [00:04, 13.55it/s]
66it [00:04, 13.73it/s]
68it [00:05, 13.37it/s]
70it [00:05, 14.71it/s]
72it [00:05, 15.68it/s]
74it [00:05, 14.45it/s]
76it [00:05, 14.42it/s]
78it [00:05, 13.70it/s]
80it [00:0


Best Model Saved !!


Epoch: 01
	Train Total Loss: 0.295 | Train F1: 0.479
	Val. Total Loss: 0.271 | Valid F1: 0.478
_________________________________________________________________



5it [00:00, 19.14it/s]
7it [00:00, 19.03it/s]
10it [00:00, 19.66it/s]
13it [00:00, 19.49it/s]
15it [00:00, 16.70it/s]
17it [00:00, 15.56it/s]
19it [00:01, 15.06it/s]
21it [00:01, 14.14it/s]
23it [00:01, 14.20it/s]
25it [00:01, 13.71it/s]
27it [00:01, 14.13it/s]
29it [00:01, 13.61it/s]
31it [00:01, 14.13it/s]
33it [00:02, 13.47it/s]
35it [00:02, 13.82it/s]
37it [00:02, 13.33it/s]
39it [00:02, 13.44it/s]
41it [00:02, 13.19it/s]
43it [00:02, 13.43it/s]
45it [00:03, 13.21it/s]
47it [00:03, 13.20it/s]
49it [00:03, 13.43it/s]
51it [00:03, 13.15it/s]
53it [00:03, 13.79it/s]
55it [00:03, 13.29it/s]
57it [00:03, 13.79it/s]
59it [00:04, 13.28it/s]
61it [00:04, 13.75it/s]
63it [00:04, 13.30it/s]
65it [00:04, 13.65it/s]
68it [00:04, 15.40it/s]
70it [00:04, 14.45it/s]
72it [00:04, 14.42it/s]
74it [00:05, 14.06it/s]
76it [00:05, 13.76it/s]
78it [00:05, 13.61it/s]
80it [00:05, 13.49it/s]
82it [00:05, 13.65it/s]
84it [00:05, 13.32it/s]
86it [00:05, 13.92it/s]
88it [00:06, 13.34it/s]
90it [00:06, 13.8


Best Model Saved !!


Epoch: 02
	Train Total Loss: 0.259 | Train F1: 0.487
	Val. Total Loss: 0.259 | Valid F1: 0.484
_________________________________________________________________



4it [00:00, 17.87it/s]
6it [00:00, 16.66it/s]
8it [00:00, 15.10it/s]
10it [00:00, 15.27it/s]
12it [00:00, 14.15it/s]
14it [00:00, 14.13it/s]
16it [00:01, 13.63it/s]
18it [00:01, 14.01it/s]
20it [00:01, 13.51it/s]
22it [00:01, 13.66it/s]
24it [00:01, 13.35it/s]
26it [00:01, 13.36it/s]
28it [00:02, 13.08it/s]
30it [00:02, 13.39it/s]
32it [00:02, 13.02it/s]
34it [00:02, 13.52it/s]
36it [00:02, 13.20it/s]
38it [00:02, 13.55it/s]
40it [00:02, 13.31it/s]
42it [00:03, 13.30it/s]
44it [00:03, 14.79it/s]
47it [00:03, 16.40it/s]
49it [00:03, 16.37it/s]
51it [00:03, 14.81it/s]
53it [00:03, 14.62it/s]
55it [00:03, 13.93it/s]
57it [00:04, 13.99it/s]
59it [00:04, 13.93it/s]
61it [00:04, 14.07it/s]
63it [00:04, 13.60it/s]
65it [00:04, 13.75it/s]
67it [00:04, 13.37it/s]
69it [00:04, 13.64it/s]
71it [00:05, 13.28it/s]
73it [00:05, 13.39it/s]
75it [00:05, 13.40it/s]
77it [00:05, 13.12it/s]
79it [00:05, 13.32it/s]
81it [00:05, 13.11it/s]
83it [00:05, 13.85it/s]
85it [00:06, 13.38it/s]
87it [00:06, 14.01

...


Best Model Saved !!


Epoch: 08
	Train Total Loss: 0.234 | Train F1: 0.596
	Val. Total Loss: 0.241 | Valid F1: 0.591
_________________________________________________________________



5it [00:00, 20.03it/s]
7it [00:00, 19.74it/s]
10it [00:00, 19.65it/s]
12it [00:00, 19.27it/s]
14it [00:00, 19.13it/s]
17it [00:00, 19.47it/s]
19it [00:00, 19.44it/s]
21it [00:01, 19.46it/s]
23it [00:01, 19.02it/s]
25it [00:01, 18.84it/s]
27it [00:01, 18.63it/s]
29it [00:01, 16.84it/s]
31it [00:01, 15.60it/s]
33it [00:01, 14.77it/s]
35it [00:01, 14.56it/s]
37it [00:02, 13.84it/s]
39it [00:02, 13.99it/s]
41it [00:02, 13.47it/s]
43it [00:02, 13.98it/s]
45it [00:02, 13.53it/s]
47it [00:02, 14.18it/s]
50it [00:03, 15.97it/s]
52it [00:03, 15.59it/s]
54it [00:03, 14.43it/s]
56it [00:03, 14.39it/s]
58it [00:03, 13.71it/s]
60it [00:03, 14.23it/s]
62it [00:03, 13.58it/s]
64it [00:04, 14.15it/s]
66it [00:04, 13.48it/s]
68it [00:04, 13.88it/s]
70it [00:04, 13.50it/s]
72it [00:04, 13.53it/s]
74it [00:04, 13.50it/s]
76it [00:04, 13.35it/s]
78it [00:05, 13.52it/s]
80it [00:05, 13.18it/s]
82it [00:05, 13.82it/s]
84it [00:05, 13.39it/s]
86it [00:05, 13.72it/s]
88it [00:05, 13.21it/s]
90it [00:05, 13.3



Epoch: 09
	Train Total Loss: 0.234 | Train F1: 0.587
	Val. Total Loss: 0.249 | Valid F1: 0.633
_________________________________________________________________



4it [00:00, 13.11it/s]
6it [00:00, 12.90it/s]
8it [00:00, 13.40it/s]
10it [00:00, 13.07it/s]
12it [00:00, 13.47it/s]
14it [00:01, 13.45it/s]
16it [00:01, 13.35it/s]
18it [00:01, 13.36it/s]
20it [00:01, 13.12it/s]
22it [00:01, 13.50it/s]
24it [00:01, 13.15it/s]
26it [00:01, 13.74it/s]
28it [00:02, 13.14it/s]
30it [00:02, 13.52it/s]
32it [00:02, 13.17it/s]
34it [00:02, 13.75it/s]
36it [00:02, 13.25it/s]
38it [00:02, 13.62it/s]
40it [00:02, 13.26it/s]
42it [00:03, 13.56it/s]
44it [00:03, 13.62it/s]
46it [00:03, 13.29it/s]
48it [00:03, 13.67it/s]
50it [00:03, 13.29it/s]
52it [00:03, 13.56it/s]
54it [00:04, 13.20it/s]
56it [00:04, 13.78it/s]
58it [00:04, 13.37it/s]
60it [00:04, 13.88it/s]
62it [00:04, 13.39it/s]
64it [00:04, 13.72it/s]
66it [00:04, 14.17it/s]
68it [00:05, 14.41it/s]
70it [00:05, 14.53it/s]
73it [00:05, 16.16it/s]
75it [00:05, 14.73it/s]
77it [00:05, 14.64it/s]
79it [00:05, 13.95it/s]
81it [00:05, 13.91it/s]
83it [00:06, 13.75it/s]
85it [00:06, 13.56it/s]
87it [00:06, 13.89


Best Model Saved !!


Epoch: 10
	Train Total Loss: 0.230 | Train F1: 0.592
	Val. Total Loss: 0.240 | Valid F1: 0.612
_________________________________________________________________
total time taken: 9.659982252120972


In [105]:
#RoBERTa
import time
from tqdm import tqdm
t1 = time.time()
try:
    for epoch in tqdm(range(N_EPOCHS)): 

        tr_l, tr_f1= train_roberta(model, train_datalaoder, optim, criterion)
        tot_t_loss.append(tr_l)

        val_l, val_f1, pred, tgt = evaluate_roberta(model, test_datalaoder, criterion)
        tot_v_loss.append(val_l)

        if val_l < best_valid_loss:
            best_valid_loss = val_l
            best_pred, best_tgt = pred, tgt
            torch.save(model.state_dict(), 'model_least_loss.pt')
            print("\nBest Model Saved !!")
        elif epoch % 3 == 0:
            torch.save(model.state_dict(), 'model_checkpoint_'+str(epoch)+'.pt')
            print("\Checkpoint Model Saved !!")
        print("\n")
        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Total Loss: {tr_l:.3f} | Train F1: {tr_f1:.3f}')
        print(f'\tVal. Total Loss: {val_l:.3f} | Valid F1: {val_f1:.3f}')
        print("_________________________________________________________________")
except e:
    print('in exception')
    print(e)
    t2 = time.time()
t2 = time.time()
print(f"total time taken: {(t2-t1)/60}")

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
2it [00:00, 11.63it/s]
4it [00:00, 12.47it/s]
6it [00:00, 12.31it/s]
8it [00:00, 13.05it/s]
10it [00:00, 12.72it/s]
12it [00:00, 13.37it/s]
14it [00:01, 12.99it/s]
16it [00:01, 13.45it/s]
18it [00:01, 13.40it/s]
20it [00:01, 13.33it/s]
22it [00:01, 13.23it/s]
24it [00:01, 13.08it/s]
26it [00:01, 13.31it/s]
28it [00:02, 13.00it/s]
30it [00:02, 13.19it/s]
32it [00:02, 12.84it/s]
34it [00:02, 13.06it/s]
36it [00:02, 12.85it/s]
38it [00:02, 13.07it/s]
40it [00:03, 12.87it/s]
42it [00:03, 13.27it/s]
44it [00:03, 12.92it/s]
46it [00:03, 13.20it/s]
48it [00:03, 12.92it/s]
50it [00:03, 13.47it/s]
52it [00:03, 13.00it/s]
54it [00:04, 13.36it/s]
56it [00:04, 12.91it/s]
58it [00:04, 13.53it/s]
60it [00:04, 13.04it/s]
62it [00:04, 13.63it/s]
64it [00:04, 13.19it/s]
66it [00:05, 13.46it/s]
68it [00:05, 13.21it/s]
70it [00:05, 13.32it/s]
72it [00:05, 13.14it/s]
74it [00:05, 13.09it/s]
76it [00:05, 13.34it/s]
78it [00:05, 12.98it/s]
80it [00:0


Best Model Saved !!


Epoch: 01
	Train Total Loss: 0.671 | Train F1: 0.456
	Val. Total Loss: 0.578 | Valid F1: 0.476
_________________________________________________________________



4it [00:00, 13.49it/s]
6it [00:00, 13.07it/s]
8it [00:00, 13.32it/s]
10it [00:00, 13.00it/s]
12it [00:00, 13.63it/s]
14it [00:01, 13.18it/s]
16it [00:01, 13.63it/s]
18it [00:01, 13.07it/s]
20it [00:01, 13.27it/s]
22it [00:01, 12.87it/s]
24it [00:01, 13.19it/s]
26it [00:01, 12.78it/s]
28it [00:02, 13.06it/s]
30it [00:02, 12.83it/s]
32it [00:02, 13.33it/s]
34it [00:02, 12.93it/s]
36it [00:02, 13.41it/s]
38it [00:02, 13.09it/s]
40it [00:03, 13.25it/s]
42it [00:03, 13.12it/s]
44it [00:03, 13.09it/s]
46it [00:03, 13.33it/s]
48it [00:03, 13.00it/s]
50it [00:03, 13.21it/s]
52it [00:03, 12.85it/s]
54it [00:04, 13.33it/s]
56it [00:04, 12.90it/s]
58it [00:04, 13.46it/s]
60it [00:04, 13.16it/s]
62it [00:04, 13.74it/s]
64it [00:04, 13.28it/s]
66it [00:04, 13.69it/s]
69it [00:05, 15.47it/s]
71it [00:05, 15.46it/s]
73it [00:05, 14.40it/s]
75it [00:05, 14.56it/s]
77it [00:05, 13.66it/s]
79it [00:05, 14.13it/s]
81it [00:06, 13.45it/s]
83it [00:06, 13.95it/s]
85it [00:06, 13.36it/s]
87it [00:06, 13.69


Best Model Saved !!


Epoch: 02
	Train Total Loss: 0.656 | Train F1: 0.400
	Val. Total Loss: 0.539 | Valid F1: 0.478
_________________________________________________________________



4it [00:00, 12.62it/s]
6it [00:00, 12.70it/s]
8it [00:00, 12.99it/s]
10it [00:00, 12.94it/s]
12it [00:00, 12.98it/s]
14it [00:01, 13.00it/s]
16it [00:01, 12.86it/s]
18it [00:01, 13.16it/s]
20it [00:01, 12.91it/s]
22it [00:01, 13.48it/s]
24it [00:01, 13.06it/s]
26it [00:01, 13.22it/s]
28it [00:02, 12.98it/s]
30it [00:02, 13.56it/s]
32it [00:02, 13.14it/s]
34it [00:02, 13.76it/s]
36it [00:02, 13.19it/s]
38it [00:02, 13.57it/s]
40it [00:03, 13.12it/s]
42it [00:03, 13.57it/s]
45it [00:03, 15.26it/s]
47it [00:03, 14.31it/s]
49it [00:03, 14.15it/s]
51it [00:03, 13.76it/s]
53it [00:03, 13.57it/s]
55it [00:04, 13.45it/s]
57it [00:04, 13.15it/s]
59it [00:04, 13.51it/s]
61it [00:04, 13.07it/s]
63it [00:04, 13.75it/s]
65it [00:04, 13.16it/s]
67it [00:04, 13.73it/s]
69it [00:05, 13.32it/s]
71it [00:05, 13.69it/s]
73it [00:05, 13.12it/s]
75it [00:05, 13.42it/s]
77it [00:05, 12.99it/s]
79it [00:05, 13.35it/s]
81it [00:06, 13.19it/s]
83it [00:06, 13.23it/s]
85it [00:06, 13.07it/s]
87it [00:06, 13.10

...



Epoch: 08
	Train Total Loss: 0.599 | Train F1: 0.582
	Val. Total Loss: 0.533 | Valid F1: 0.598
_________________________________________________________________



4it [00:00, 13.24it/s]
6it [00:00, 12.86it/s]
8it [00:00, 13.13it/s]
10it [00:00, 12.81it/s]
12it [00:00, 13.08it/s]
14it [00:01, 12.74it/s]
16it [00:01, 13.26it/s]
18it [00:01, 12.99it/s]
20it [00:01, 13.33it/s]
22it [00:01, 13.31it/s]
24it [00:01, 13.24it/s]
26it [00:01, 13.58it/s]
28it [00:02, 13.15it/s]
30it [00:02, 14.46it/s]
33it [00:02, 15.13it/s]
35it [00:02, 14.11it/s]
37it [00:02, 14.05it/s]
39it [00:02, 13.41it/s]
41it [00:03, 13.80it/s]
43it [00:03, 13.21it/s]
45it [00:03, 13.79it/s]
47it [00:03, 13.24it/s]
49it [00:03, 13.81it/s]
51it [00:03, 13.21it/s]
53it [00:03, 13.65it/s]
55it [00:04, 13.40it/s]
57it [00:04, 13.33it/s]
59it [00:04, 13.30it/s]
61it [00:04, 13.23it/s]
63it [00:04, 13.26it/s]
65it [00:04, 13.17it/s]
67it [00:04, 13.34it/s]
69it [00:05, 13.04it/s]
71it [00:05, 13.45it/s]
73it [00:05, 13.03it/s]
75it [00:05, 13.27it/s]
77it [00:05, 13.00it/s]
79it [00:05, 13.52it/s]
81it [00:06, 13.18it/s]
83it [00:06, 13.42it/s]
85it [00:06, 12.94it/s]
87it [00:06, 13.32



Epoch: 09
	Train Total Loss: 0.592 | Train F1: 0.607
	Val. Total Loss: 0.526 | Valid F1: 0.602
_________________________________________________________________



4it [00:00, 12.91it/s]
6it [00:00, 12.67it/s]
8it [00:00, 13.26it/s]
10it [00:00, 12.86it/s]
12it [00:00, 13.53it/s]
14it [00:01, 13.13it/s]
16it [00:01, 13.67it/s]
18it [00:01, 13.24it/s]
20it [00:01, 13.59it/s]
22it [00:01, 13.38it/s]
24it [00:01, 13.47it/s]
27it [00:01, 15.08it/s]
29it [00:02, 14.87it/s]
31it [00:02, 14.02it/s]
33it [00:02, 13.90it/s]
35it [00:02, 13.39it/s]
37it [00:02, 13.75it/s]
39it [00:02, 13.19it/s]
41it [00:02, 13.68it/s]
43it [00:03, 13.11it/s]
45it [00:03, 13.30it/s]
47it [00:03, 12.90it/s]
49it [00:03, 13.29it/s]
51it [00:03, 12.88it/s]
53it [00:03, 13.13it/s]
55it [00:04, 12.83it/s]
57it [00:04, 13.16it/s]
59it [00:04, 12.78it/s]
61it [00:04, 13.45it/s]
63it [00:04, 13.03it/s]
65it [00:04, 13.47it/s]
67it [00:04, 13.22it/s]
69it [00:05, 13.18it/s]
71it [00:05, 12.96it/s]
73it [00:05, 13.19it/s]
75it [00:05, 13.04it/s]
77it [00:05, 12.92it/s]
79it [00:05, 13.20it/s]
81it [00:06, 12.86it/s]
83it [00:06, 13.07it/s]
85it [00:06, 12.80it/s]
87it [00:06, 13.02

\Checkpoint Model Saved !!


Epoch: 10
	Train Total Loss: 0.587 | Train F1: 0.625
	Val. Total Loss: 0.530 | Valid F1: 0.605
_________________________________________________________________
total time taken: 4.186001201470693


In [ ]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 10 or less; roberta; positive tweets = negative tweets; no preprocessing

In [ ]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 40; roberta; positive tweets = negative tweets; no preprocessing

In [ ]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 5; roberta; 2*positive tweets = negative tweets; no preprocessing

In [106]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 10; roberta; 2*positive tweets = negative tweets; no preprocessing

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      4660
           1       0.27      0.33      0.30       474

    accuracy                           0.85      5134
   macro avg       0.60      0.62      0.61      5134
weighted avg       0.87      0.85      0.86      5134



In [ ]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 10; roberta; 3*positive tweets = negative tweets; no preprocessing

In [ ]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 20; roberta; 3*positive tweets = negative tweets; no preprocessing

In [78]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 10; BERT; entire; basic preprocessing

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      4660
           1       0.55      0.18      0.27       474

    accuracy                           0.91      5134
   macro avg       0.73      0.58      0.61      5134
weighted avg       0.89      0.91      0.89      5134



In [31]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 7; BERT; 2*positive = negative tweets; basic preprocessing

              precision    recall  f1-score   support

           0       0.94      0.87      0.90      4660
           1       0.25      0.43      0.32       474

    accuracy                           0.83      5134
   macro avg       0.59      0.65      0.61      5134
weighted avg       0.87      0.83      0.85      5134



In [53]:
print(metrics.classification_report(best_tgt, best_pred))
#epoch 7; BERTweet; entire data; basic preprocessing

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      4660
           1       0.64      0.12      0.20       474

    accuracy                           0.91      5134
   macro avg       0.78      0.56      0.58      5134
weighted avg       0.89      0.91      0.88      5134

